In [421]:
import pandas as pd
import numpy as np

In [422]:
df = pd.read_csv('data/CustChurn.csv')
df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)
df["TotalCharges"] = df["TotalCharges"].astype(float)
df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].mean())
df.drop("customerID", axis=1, inplace=True)
cols = [
    "StreamingTV",
    "StreamingMovies",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
]
df[cols] = df[cols].replace("No internet service", "No")
df["MultipleLines"] = df["MultipleLines"].replace("No phone service", "No")
df["Churn"] = df["Churn"].map({"Yes":1, "No":0})
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,Male,0,No,No,45,No,No,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,0
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,0
7040,Female,0,Yes,Yes,11,No,No,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,0
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,1


In [423]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [424]:
X

array([['Female', 0, 'Yes', ..., 'Electronic check', 29.85, 29.85],
       ['Male', 0, 'No', ..., 'Mailed check', 56.95, 1889.5],
       ['Male', 0, 'No', ..., 'Mailed check', 53.85, 108.15],
       ...,
       ['Female', 0, 'Yes', ..., 'Electronic check', 29.6, 346.45],
       ['Male', 1, 'Yes', ..., 'Mailed check', 74.4, 306.6],
       ['Male', 0, 'No', ..., 'Bank transfer (automatic)', 105.65,
        6844.5]], shape=(7043, 19), dtype=object)

In [425]:
y

array([0, 0, 1, ..., 0, 1, 0], shape=(7043,))

In [426]:
for i in range(0, 19):
    print(X[0, i], end=" ")

Female 0 Yes No 1 No No DSL No Yes No No No No Month-to-month Yes Electronic check 29.85 29.85 

In [427]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
yes_no_col = [0,2,3,5,6,8,9,10,11,12,13,15]
for col in yes_no_col:
    X[:, col] = le.fit_transform(X[:, col])

In [428]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(), [7,-3,-5])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 29.85, 29.85],
       [1.0, 0.0, 0.0, ..., 0, 56.95, 1889.5],
       [1.0, 0.0, 0.0, ..., 1, 53.85, 108.15],
       ...,
       [1.0, 0.0, 0.0, ..., 1, 29.6, 346.45],
       [0.0, 1.0, 0.0, ..., 1, 74.4, 306.6],
       [0.0, 1.0, 0.0, ..., 1, 105.65, 6844.5]],
      shape=(7043, 26), dtype=object)

In [429]:
X = X.astype('float32')
y = y.astype('float32')

In [430]:
X.shape

(7043, 26)

In [431]:
for i in range(0, 26):
    print(X[0, i], end=" ")

1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 29.85 29.85 

In [432]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [433]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [434]:
import tensorflow as tf
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

In [435]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

In [436]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

In [437]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [438]:
ann.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])

In [439]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 332us/step - accuracy: 0.7158 - loss: 0.5810
Epoch 2/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.7336 - loss: 0.4669
Epoch 3/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7696 - loss: 0.4451
Epoch 4/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.7945 - loss: 0.4343
Epoch 5/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - accuracy: 0.7959 - loss: 0.4286
Epoch 6/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step - accuracy: 0.7959 - loss: 0.4236
Epoch 7/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step - accuracy: 0.8019 - loss: 0.4201
Epoch 8/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 374us/step - accuracy: 0.8033 - loss: 0.4170
Epoch 9/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.8049 - loss: 0.4158
Epoch 10/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.8062 - loss: 0.4140
Epoch 11/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step - accuracy: 0.8081 - loss: 0.4126
Epoch 12/100
177/17

In [440]:
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 1.]
 [0. 0.]
 [0. 0.]]


In [441]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[923 118]
 [184 184]]


0.7856635911994322